# Reconocimiento de Flores

## Práctica Final - Visión por computador
---
##### Autores: Marta Gómez Macías, Braulio Vargas López
---

### Definición del problema

El problema elegido consiste en extraer una serie de características del conjunto de imágenes para ser capaces de reconocer la especie de flor que es de las 17 especies posibles, mediante una imagen. Este problema consta de varias partes:

1. __Extracción de las características principales de la imagen__.
2. __Entrenar un algoritmo de aprendizaje a partir de las características extraídas__.

Para esto, vamos a hacer uso de distintos descriptores, como son el descriptor *HOG*, SIFT, entre otros. Una vez tengamos los descriptores, utilizaremos la técnica _Bag of Words_ para obtener un descriptor que nos sirva para obtener un modelo de aprendizaje, capaz de clasificar las distintas clases de flores.

La técncia _Bag Of Words_ consiste en extraer una serie de características de las imágenes, conocidas como palabras. Cada imagen, tendrá una serie de palabras que la identifican y con estas palabras, podremos construir un vocabulario que almacena todas las palabras extraídas de todas las imágenes.

Con este vocabulario, podremos hacer un histograma que nos indica con qué fuerza vota o no una imagen a dichas palabras del vocabulario, con lo que obtendremos un vector de características de esa imagen, útil para el algoritmo de aprendizaje, y que no deja de tener relación con la propia imagen.

Este vocabulario lo podemos construir haciendo uso de técnicas que se centran en la geometría de la imagen y son invariantes a la escena, como son los descriptores de SIFT, SURF, etc. Pero, también es posible utilizar otro tipo de descritores como son el ya mencionado, HOG.

El descriptor HOG, conocido como ___Histogram of Oriented Gradientes___ es muy utilizado para la detección de objetos. Este descriptor computa un histograma de valores donde se indican la frecuencia con la que se dan las direcciones de los gradientes en pequeñas divisiones de la imagen.

### Enfoque de la implementación y eficiencia.

#### 1: Extracción de las características principales de la imagen

En este primer paso, hemos implementado una función que calcula varios tipos de descriptores de una imagen, según el tipo de detector que queramos usar. Según el tipo de descriptor, generará un detector que irá construyendo un vocabulario, a partir de los `KeyPoints` detectados y su descriptor.

Este vocabulario se inicializa como un ```BOWKMeansTrainer``` y con la función ```add``` vamos añadiendo los descriptores obtenidos.

#### 2. Clusterizar los descriptores obtenidos

Para obtener los clusters, al haber creado el vocabulario con ```BOWKMeansTrainer```, podemos usar la función ```cluster``` para realizar el clustering con _K-Medias_ y así obtener el vocabulario clusterizado, listo para obtener los descriptores de cada imagen.

Esta función de OpenCV, ya implementa el método de clustering utilizando multiprocesamiento en paralelo, para aumentar la eficiencia del algoritmo.

#### 3. Obtener los histogramas de cada imagen

Una vez construida la bolsa de palabras, tenemos ver con qué fuerza vota cada una de las imágenes para las distintas palabras que hemos extraído. Para ello, utilizaremos la función de OpenCV ```BOWImgDescriptorExtractor```, que necesita un detector de puntos, que crearemos del mismo tipo que se usó para la construcción del vocabulario, y un *matcher*. En este caso, usaremos un matcher basado en fuerza fuerza bruta. 

Con todo esto, podremos calcular el histograma para cada imagen y obtener el descriptor para cada una de ellas, haciendo uso de la función de OpenCV ```compute``` que utiliza la imagen y el detector para obtener los *keypoints* y computar el histograma.

#### Implementación del modelo de aprendizaje
Para realizar los modelos de aprendizaje, hemos usado la librería `ml` de _OpenCV_, más concretamente hemos entrenado un modelo _Support Vector Machine_ y un modelo _Random Forest_. A la hora de dividir los datos en conjunto de training y test, hemos procurado que haya, al menos, una imagen de cada clase en ambos conjuntos.

In [ ]:
def create_train_subset():
    subset = []
    for i in range(0,1360,80):
        subset += np.random.randint(low=i,high=i+80,size=4).tolist()

    return np.array(subset)


test_mask = create_train_subset()
aux = np.arange(1360)
training_mask = np.in1d(aux, test_mask) * 1
training_mask = np.where(training_mask == 0)[0]